# Scrapping


In [55]:
from selenium import webdriver

import requests
import time
import pandas as pd
import random
from bs4 import BeautifulSoup

url="https://itch.io/games/top-rated"

Script for scrolling down the page

In [ ]:
browser = webdriver.Firefox()
browser.get(url)

i=0
j=3631
while i<j:
    browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(2)
    i=i+1



soup=BeautifulSoup(browser.page_source,"html.parser")
browser.close()

Extracting titles, links, authors and prices

In [ ]:
game_url=[]
game_author=[]
game_title=[]
game_price=[]
games =soup.findAll('div',{"class":"game_cell_data"})
for game in games:
    try:
        title=game.find('a',{"class":"title"}).get_text()
    except:
        title=''
    try:
        price=game.find('div',{'class':'price_value'}).get_text()
    except:
        price=""
    try:
        author=game.find('div',{"class":"game_author"}).get_text()
    except:
           author=""
    try:
        urls=game.find('a')['href']
    except:
        urls=''

    game_title.append(title)
    game_price.append(price)
    game_author.append(author)
    game_url.append(urls)

Creating a dataframe

In [ ]:

d={'title':game_title,'author':game_author,'price':game_price,'url':game_url}
df=pd.DataFrame(d)


In [ ]:
df.to_csv('game_data.csv', sep='\t',index=False)

In [ ]:
df1 = pd.read_csv('game_data.csv',error_bad_lines=False,sep='\t')


Scrapping the data from the game pages

In [ ]:
game_status=[]
game_platform=[]
game_rating=[]
made_with=[]
game_tags=[]
game_accessibility=[]
game_ASL=[]

for g_url in df1.url:
    time.sleep(random.randint(1, 5))
    g_html=requests.get(g_url)
    game_soup=BeautifulSoup(g_html.content,"html.parser")


    g_status=''
    g_platform=''
    g_rating=''
    g_made_with=''
    g_tags=''
    g_accessibility=''
    g_ASL=''
    i=0
    try:
        tbl=game_soup('table')[i]
    except:
        1
    else:
        tbl=game_soup('table')[i]
        tbl_rows = tbl.findAll('tr')
        for rows in tbl_rows:
            try:
                rows.find('td').get_text()
            except:
                i=i+1
                tbl=game_soup('table')[i]
            else:
                if rows.find('td').get_text()=='Status':
                    g_status=rows.findAll('td')[1].get_text()
                if rows.find('td').get_text()=='Platforms':
                    g_platform=rows.findAll('td')[1].get_text()
                if rows.find('td').get_text()=='Rating':
                    g_rating=(game_soup.find("div", {"class": "aggregate_rating"})["title"])
                if rows.find('td').get_text()=='Made with':
                    g_made_with=rows.findAll('td')[1].get_text()
                if rows.find('td').get_text()=='Tags':
                    g_tags=rows.findAll('td')[1].get_text()
                if rows.find('td').get_text()=='Accessibility':
                    g_accessibility=rows.findAll('td')[1].get_text()
                if rows.find('td').get_text()=='ASL':
                    g_ASL=rows.findAll('td')[1].get_text()
    finally:
        game_status.append(g_status)
        game_platform.append(g_platform)
        game_rating.append(g_rating)
        made_with.append(g_made_with)
        game_tags.append(g_tags)
        game_accessibility.append(g_accessibility)
        game_ASL.append(g_ASL)


Finishing the dataframe

In [ ]:
df1['status'] = game_status
df1['platform'] = game_platform
df1['rating'] = game_rating
df1['made_with'] = made_with
df1['tags'] = game_tags
df1['accessibility'] = game_accessibility
df1['ASL'] = game_ASL
df1.to_csv('final_data.csv', sep='\t',index=False)





